In [1]:
import glob
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.task import def_task,task_constants
from collections import defaultdict
from typing import Dict
import logging
from scipy import fft,signal

In [2]:
def get_fr_df(filepath,in_out,cgroup,e_align):
    py_f = np.load(filepath,allow_pickle=True).item(0)
    sp = py_f['sp_data']
    bhv = py_f['bhv']
    trial_idx = select_trials.select_trials_block(sp, n_block=1)
    trial_idx = select_trials.select_correct_trials(bhv, trial_idx)
    task = def_task.create_task_frame(trial_idx, bhv, task_constants.SAMPLES_COND)
    neurons = np.where(sp["clustersgroup"] == cgroup)[0]
    fr_samples = firing_rate.fr_by_sample_neuron(
            sp=sp,
            neurons=neurons,
            task=task,
            in_out=in_out,
            kernel=0,
            e_align=e_align,
            plot=False
        )
    return fr_samples

In [91]:
# load all pfc files
pfc_path = "/home/INT/losada.c/Documents/data/test/areas/pfc/*"
list = glob.glob(pfc_path)
cgroup = "good"
in_out=1
e_align=2
win_size = 100
step = 1
fix_duration = 200
sample_duration = 450
fr_samples=[]
max_n_trials = []
num_neurons,num_trials = 0,0
for n_path, path in enumerate(list):
    print(path)
    df= get_fr_df(path,in_out,cgroup,e_align)
    rolling_df = df.loc[:,:'neuron'].iloc[:,:-1].rolling(window = win_size,axis=1,step=step,min_periods=1).mean()
    rolling_df = rolling_df.iloc[:,df['sample_on'][0]-fix_duration:df['sample_on'][0]+sample_duration]
    rolling_df.columns = np.arange(sample_duration+fix_duration)
    rolling_df = pd.concat([rolling_df,df[['neuron','sample','trial_idx']]],axis=1)
    # max number of trials that can be used 
    max_n_trials.append(rolling_df[['neuron','sample']][rolling_df['neuron']==1].groupby(['sample']).count().min()[0])
    # rename neurons
    unique_neurons = rolling_df['neuron'].unique()
    rolling_df['neuron'].replace(unique_neurons,np.arange(num_neurons,num_neurons+len(unique_neurons)),inplace=True)
    num_neurons += len(unique_neurons)
    # rename trials
    unique_trials = rolling_df['trial_idx'].unique()
    rolling_df['trial_idx'].replace(unique_trials,np.arange(len(unique_trials)),inplace=True)
    num_trials += len(unique_trials)
    fr_samples.append(rolling_df)
all_df = pd.concat(fr_samples)

/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-22_10-59-03_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1308
Number of correct trials in block 2: 574
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-12-14_10-50-49_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1753
Number of correct trials in block 2: 874
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-30_10-42-03_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1588
Number of correct trials in block 2: 769
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-11-28_10-23-27_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1536
Number of correct trials in block 2: 657
/home/INT/losada.c/Documents/data/test/areas/pfc/2022-12-16_10-30-59_Riesling_pfc_e1_r1.npy
Number of trials in block 1: 1340
Number of correct trials in block 2: 582


In [125]:
# iterar por sample y con esto me armo el dataset para train and test (t=0, esto mismo se repite para cada t)
n_df = all_df[all_df['sample']=='o0_c0']
data = n_df[[0,'neuron','trial_idx']]
n_pivot = pd.pivot_table(data, values=0,index='trial_idx',columns='neuron').reset_index(drop=True).loc[:,0:]
pp = []
for n in np.arange(len(n_pivot.columns)):
    pp.append(n_pivot.loc[~n_pivot[n].isna(),n].sample(min(max_n_trials),random_state=1).reset_index(drop=True))
pd.concat(pp,axis=1)

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0.00,0.00,0.01,0.01,0.00,0.00,0.02,0.03,0.00,0.01,...,0.02,0.00,0.00,0.02,0.05,0.00,0.00,0.01,0.01,0.00
1,0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,...,0.02,0.00,0.00,0.00,0.02,0.04,0.00,0.02,0.02,0.00
2,0.05,0.03,0.01,0.00,0.00,0.00,0.01,0.01,0.01,0.02,...,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00
3,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,...,0.01,0.02,0.00,0.00,0.02,0.01,0.00,0.01,0.00,0.00
4,0.06,0.02,0.03,0.01,0.00,0.00,0.00,0.02,0.00,0.06,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.02,0.00,0.02,0.01,0.00,0.03,0.00,0.03,0.01,0.00,...,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.00
6,0.06,0.01,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.04,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02,...,0.00,0.02,0.00,0.00,0.06,0.02,0.00,0.04,0.01,0.00
8,0.08,0.03,0.03,0.00,0.00,0.01,0.01,0.03,0.01,0.02,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
9,0.08,0.01,0.03,0.00,0.01,0.00,0.02,0.00,0.00,0.03,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.00


### SVM

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV,KFold,cross_val_score,StratifiedKFold
from sklearn.svm import SVC,LinearSVC
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics

In [ ]:
def run_svm_decoder(model,list_df,n_iterations,steps,win_steps,win_size,it_seed,resample_value):
    scores = np.zeros((n_iterations,steps))
    for n_it in np.arange(n_iterations):
        #  select trials randomly
        df = []
        for i_df in list_df:
            df.append(i_df.sample(resample_value,random_state=it_seed[n_it],replace=False)) # sample without replacement
        df = pd.concat(df).reset_index(drop=True)
        
        #df = df.sample(frac=1,random_state=it_seed[n_it],replace=False)
        
        X = df.iloc[:, :-4] 
        #X = pd.DataFrame(scaler.fit_transform(X))
        y = df['label_encoder']
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2,shuffle=True, random_state=it_seed[n_it])
        X_train['label_encoder'] = y_train
        #X_test['label_encoder'] = y_test
        # Sample with replacement
        X_train = X_train.sample(len(X_train),random_state=it_seed[n_it],replace=False)
        y_train = X_train['label_encoder']
        X_train = X_train.iloc[:,:-1]
        for n,n_win in enumerate(np.arange(0,steps*win_steps,win_steps)):
            # Split into X and y
            scaler = MinMaxScaler()
            X = X_train.iloc[:,int(n_win):int(n_win)+win_size].mean(axis=1).values.reshape(-1, 1)
            X = pd.DataFrame(scaler.fit_transform(X))    
            # test
            X_t = X_test.iloc[:,int(n_win):int(n_win)+win_size].mean(axis=1).values.reshape(-1, 1)        
            X_t = pd.DataFrame(scaler.transform(X_t))
            model.fit(X,y_train)
            y_predict = model.predict(X_t)
            scores[n_it, n] = metrics.accuracy_score(y_test,y_predict)# cross_val_score(model, X, y, cv=kf,n_jobs=-1).mean()
    return scores

In [ ]:
# plot results
def plot_accuracy(scores,win_steps,neuron_max_shift,x_lim_min,x_lim_max,n_neuron):
    fig,ax = plt.subplots()
    ax.plot(((np.arange(0,len(scores[0]))*win_steps)- neuron_max_shift[n_neuron-1])/1000,scores[:13].mean(axis=0))
    ax.set_xlim(x_lim_min, x_lim_max)
    ax.vlines(
        0, 0.3, 1, color="k", linestyles="dashed"
    )# sample on
    ax.hlines(
        0.5, x_lim_min, x_lim_max, color="gray", linestyles="solid"
    )
    ax.set_title('Is neuron %d engaged in the task?'%(n_neuron))
    ax.set(xlabel="Time (s)", ylabel="SVM classifier accuracy")
    fig.tight_layout(pad=0.2, h_pad=0.2, w_pad=0.2)
    fig.legend(['Accuracy','Sample on'], fontsize=9)